In [4]:
import datetime

import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution
from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import median_absolute_error

from sklearn.preprocessing import MaxAbsScaler
import math
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator, ClassifierMixin


class GRNN(BaseEstimator, ClassifierMixin):
    def __init__(self, name="GRNN", sigma=0.1):
        self.name = name
        self.sigma = 2 * np.power(sigma, 2)

    def predict(self, instance_X, train_X, train_y):
        gausian_distances = np.exp(-np.power(np.sqrt((np.square(train_X - instance_X).sum(axis=1))), 2) \
                                   / self.sigma)
        gausian_distances_sum = gausian_distances.sum()
        if gausian_distances_sum < math.pow(10, -7): gausian_distances_sum = math.pow(10, -7)
        result = np.multiply(gausian_distances, train_y).sum() / gausian_distances_sum
        return result


def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

#df_train = pd.read_csv("Nzone1_train.txt", header=None)
#df_test = pd.read_csv("Nzone1_test.txt", header=None)
df_train = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/OBDTrain.txt', header=None)
df_test = pd.read_csv('/Users/ivanizonin/izonin/sgtm/Datasets/OBDTest.txt', header=None)

train_X = df_train.iloc[:, :-1]
train_y = df_train.iloc[:, -1]
test_X = df_test.iloc[:, :-1]
test_y = df_test.iloc[:, -1]

scaler = MaxAbsScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
test_X = scaler.transform(test_X)
def f(s):
    train_y_predictions = []
    for i in range(0, len(train_X)):
        grnn = GRNN(sigma=s[0])
        train_X_1 = train_X.copy()
        train_y_1 = train_y.copy()
        train_X_1 = np.delete(train_X_1, obj=i, axis=0)
        train_y_1 = train_y_1.drop([i])
        train_y_predictions.append(grnn.predict(train_X[i], train_X_1, train_y_1))

    test_y_predictions = []
    for i in range(0, len(test_X)):
        grnn = GRNN(sigma=s[0])
        test_y_predictions.append(grnn.predict(test_X[i], train_X, train_y))

    train_X_augmented = np.empty([len(train_X) ** 2, len(train_X[0]) * 2 + 2])
    train_Z_augmented = []
    counter = 0
    for i in range(0, len(train_X)):
        for j in range(0, len(train_X)):
            train_X_augmented[counter] = np.append(np.append(np.concatenate([train_X[i], train_X[j]]),
                                                             train_y_predictions[i]),
                                                   train_y_predictions[j])
            train_Z_augmented.append(train_y_predictions[i] - train_y_predictions[j])
            counter += 1
    train_Z_augmented = pd.Series(train_Z_augmented)
    test_Z_augmented = [[0 for _ in range(0, len(train_X))] for _ in range(0, len(test_X))]
    for i in range(0, len(test_X)):
        for j in range(0, len(train_X)):
            grnn = GRNN(sigma=s[1])
            test_Z_augmented[i][j] = grnn.predict(np.append(np.append(np.concatenate([test_X[i], train_X[j]]),
                                                                          test_y_predictions[i]),
                                                                train_y_predictions[j]), train_X_augmented,
                                                      train_Z_augmented)
    test_y_final_predictions = []
    for i in range(0, len(test_Z_augmented)):
        test_y_final_predictions.append((sum(test_Z_augmented[i]) + sum(train_y_predictions)) / len(train_y))
    test_y_final_predictions = pd.Series(test_y_final_predictions)
    print("Sigma1 = " + str(s[0]))
    print("Sigma2 = " + str(s[1]))
    print(test_y_final_predictions)
    print('Testing errors:')
    print("MAPE: " + str(mean_absolute_percentage_error(test_y, test_y_final_predictions)))
    print("RMSE: " + str(root_mean_squared_error(test_y, test_y_final_predictions)))
    print("MAE: " + str(mean_absolute_error(test_y, test_y_final_predictions)))
    print("R2: " + str(r2_score(test_y, test_y_final_predictions)))
    print("MSE: " + str(mean_squared_error(test_y, test_y_final_predictions)))
    print("ME: " + str(max_error(test_y, test_y_final_predictions)))
    print("MedAE: " + str(median_absolute_error(test_y, test_y_final_predictions)))
    return -r2_score(test_y, test_y_final_predictions)
    #print(test_y_final_predictions)


start = datetime.datetime.utcnow()
res = differential_evolution(f, bounds=[(0.01, 1.5), (0.01, 4)])
end = datetime.datetime.utcnow()
print("Time: " + str((end - start).total_seconds()))

Sigma1 = 0.8454447441641035
Sigma2 = 3.054919538651455
0    16.417172
1    17.019229
2    16.447518
3    16.257180
4    16.739853
5    16.691336
6    16.771611
dtype: float64
Testing errors:
MAPE: 0.6658061335204266
RMSE: 6.891478957364608
MAE: 5.990865890945973
R2: -0.08861235026119152
MSE: 47.492482219799186
ME: 11.781335783761019
MedAE: 6.9298528753252295
Sigma1 = 0.30089032974035523
Sigma2 = 3.894713700013704
0    16.703003
1    19.286257
2    17.126470
3    14.266466
4    18.587093
5    18.896763
6    18.195935
dtype: float64
Testing errors:
MAPE: 0.7605162414095582
RMSE: 7.709795881927412
MAE: 6.3510849462453525
R2: -0.3624925887834358
MSE: 59.44095254098488
ME: 13.986763166685982
MedAE: 7.073529971324291
Sigma1 = 0.8760712909368906
Sigma2 = 0.923976241476006
0    15.693611
1    17.878382
2    15.796129
3    15.346884
4    17.401673
5    17.219290
6    17.493009
dtype: float64
Testing errors:
MAPE: 0.6982516713794816
RMSE: 7.302487975381972
MAE: 6.22270942215396
R2: -0.2223345549